In [33]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, cohen_kappa_score, balanced_accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [34]:
# read the pre-processed data all agreed
data = pd.read_csv('GS_All_Agreed.csv', index_col=0)

# generate the local dataframe for different domains
local_fruit = data.loc[data['domain_x']=='fruit'].reset_index()
local_tool = data.loc[data['domain_x']=='tool'].reset_index()
local_music = data.loc[data['domain_x']=='music'].reset_index()
local_furniture = data.loc[data['domain_x']=='furn'].reset_index()
local_garments = data.loc[data['domain_x']=='garm'].reset_index()

In [35]:
# define features and target
features = ['ngrams_last_mean',
            'nrdirhypers_x',
            'nrhypos_x',
            'nrpartrels_normalised_x',
            'depthfromtopsynset_normalised_x',
            'glosslength_normalised_x',
            'minwordlength_x',
            'nroflemmas_x',
            'polyscore_max_x']
target = ['vote_x'] # nb / b

In [36]:
# specific corpora features
# written general
# features.append('BNC_sum')
# BNCSumFeatures = features

# spoken general
# features.append('CABNC_per100,000')
# CABNC_per100KFeatures = features

# written children
# features.append('KBNC_Sum')
# KBNCSumFeatures = features

# spoken children
features.append('CHILDES_Sum_Rel')
CHILDESSumRelFeatures = features

In [37]:
# split training set and testing set using K-Flod
K = 10
random_seed = 7 # R
data = data.reset_index()
X = data[CHILDESSumRelFeatures]
y = data[target]

K_Flod = StratifiedKFold(n_splits=K, shuffle=True, random_state=random_seed)
K_Flod.get_n_splits(X, y)
cohen_kappa = []
balanced_acc = []
for train_index, test_index in K_Flod.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # SMOTE algorithm
    smote = SMOTE(random_state=random_seed, k_neighbors=2)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    # define random forest model
    rf = RandomForestClassifier(random_state=random_seed, max_features='sqrt', n_estimators=1400, min_samples_split=2, min_samples_leaf=1, max_depth=50, oob_score=True, criterion='gini', bootstrap=True).fit(X_train, y_train)

    # predict and make score
    pipeline = make_pipeline(smote, rf)
    y_pred = pipeline.predict(X_test)

    kappa = cohen_kappa_score(y_test, y_pred)
    cohen_kappa.append(kappa)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    balanced_acc.append(balanced_accuracy)

results = classification_report(y_test, y_pred, output_dict=True)
results = pd.DataFrame(results).transpose()

results['cohen kappa / 10'] = np.mean(cohen_kappa)
results['balanced acc / 10'] = np.mean(balanced_acc)
results['global'] = 5

In [38]:
results

,precision,recall,f1-score,support,cohen kappa / 10,balanced acc / 10,global
b,0.761905,0.941176,0.842105,17.000000,0.69074,0.857564,5
nb,0.983871,0.924242,0.953125,66.000000,0.69074,0.857564,5
accuracy,0.927711,0.927711,0.927711,0.927711,0.69074,0.857564,5
macro avg,0.872888,0.932709,0.897615,83.000000,0.69074,0.857564,5
weighted avg,0.938408,0.927711,0.930386,83.000000,0.69074,0.857564,5


In [39]:
# importance of features
importance = rf.feature_importances_

importance = pd.DataFrame([features, importance]).transpose()
importance = importance.rename(columns={0:'feature', 1:'importance'}).sort_values('importance', ascending=False)

In [40]:
importance

,feature,importance
4,depthfromtopsynset_normalised_x,0.362421
5,glosslength_normalised_x,0.135476
0,ngrams_last_mean,0.129221
6,minwordlength_x,0.108
3,nrpartrels_normalised_x,0.08313
9,CHILDES_Sum_Rel,0.072765
2,nrhypos_x,0.040549
8,polyscore_max_x,0.03807
7,nroflemmas_x,0.027884
1,nrdirhypers_x,0.002485
